# ComCamSim 2024 April Ops Rehearsal 3 - DIA
Author: Michael Wood-Vasey <wmwv@pitt.edu>
Last Verified to Run: 2024-04-15

Let's look at how the DIA did for the April Ops Rehearsal 3.
For some context details, see

https://confluence.lsstcorp.org/display/DM/Prompt+Processing+with+LSSTComCamSim+ops-rehearsal-3

And read Meredith Rawl's notebook at  
https://nbviewer.org/github/lsst-dm/vv-team-notebooks/blob/fab304924fd036c479fc4f61269a2041c287d146/notebooks/PREOPS-5029-DiaReasonable.ipynb

Here, I'm mostly trying to get started

1. [ ] Look at basic statistics of processing
2. [ ] Identify one calexp DIA to explore
3. [ ] Read catalog and get DiaSrc
4. [ ] Look at postage stamps, including mask plane, for 1, 10, 100 candidates

Maybe later:  
* [ ] Understand what we get from DRP vs. APDB

Run at RSP on USDF.  Run using `w_2024_15`.  I don't know what the minimum number needs to be.  In particular, the current "Recommend Weekly" is `w_2024_4`; I don't know if this is sufficient.


was helpful in getting started.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from lsst.daf.butler import Butler

In [ ]:
from astropy.visualization import ZScaleInterval, SqrtStretch, ImageNormalize, MinMaxInterval, LogStretch
from astropy.table import Table, join
import lsst.daf.butler as dafButler
from lsst.daf.butler import DatasetNotFoundError
from lsst.pipe.base import Instrument
from lsst.ap.association.transformDiaSourceCatalog import UnpackApdbFlags
import lsst.afw.display as afwDisplay
import lsst.display.astrowidgets
afwDisplay.setDefaultBackend('astrowidgets')

Key orientation information: repo, collection, dataset types.

In [ ]:
repo = "/repo/embargo"
# There were three nights for the Ops Rehearsal.
# Prompt Procesing for 2024-04-02 and 2024-04-03 did not yield good results so
# we focus on 2024-04-04.
collection = "LSSTComCamSim/prompt/output-2024-04-04"
instrument = "LSSTComCamSim"

collections = [collection, "LSSTComCamSim/templates", "LSSTComCamSim/defaults"]

Dataset Types we'll be interested in  
`postISRCCD`  (I don't think there's a "calexp" in prompt processing)  
`goodSeeingDiff_differenceExp`  
`goodSeeingDiff_diaSrc`  (or goodSeeingDiff_diaSrcTable depending on the schema you want.)

In [ ]:
butler = Butler(repo, collections=collections, instrument=instrument)

In [ ]:
registry = butler.registry
for ref in registry.queryDatasets('raw'):
    print(ref.dataId.full)

In [ ]:
registry = butler.registry
for ref in registry.queryDatasets('postISRCCD'):
    print(ref.dataId.full)

In [ ]:
registry = butler.registry
for ref in registry.queryDatasets('goodSeeingDiff_differenceExp'):
    print(ref.dataId.full)

In [ ]:
data_id = {"detector": 0, "exposure": 7024040400318}

raw = butler.get("raw", dataId=data_id)
post_isr_ccd = butler.get("postISRCCD", dataId=data_id)

In [ ]:
norm = ImageNormalize(raw.image.array, interval=ZScaleInterval(), stretch=SqrtStretch())
plt.imshow(raw.image.array, norm=norm)
plt.show()

In [ ]:
norm = ImageNormalize(post_isr_ccd.image.array, interval=ZScaleInterval(), stretch=SqrtStretch())
plt.imshow(post_isr_ccd.image.array, norm=norm)
plt.show()

In [ ]:
# Now we have to rename "exposure" -> "visit"?  Why?  This is confusing
data_id["visit"] = data_id["exposure"]

diff_exp = butler.get("goodSeeingDiff_differenceExp", dataId=data_id)
# dia_src = butler.get("goodSeeingDiff_diaSrc", dataId=data_id)

Hmmm... how do we figure out which data IDs did get processed through to DIA?

In [ ]:
# These are the runs that this chained collection is derived from
butler.registry.queryCollections(collection, flattenChains=True)